# Ejecución de la red CTC (Connectionist Temporal Classification)

## Preparación



### Instalación de paquetes necesarios







In [0]:
!pip3 install python_speech_features

### Importación del contenido del drive y configuración de la ruta de ejecución del código

In [0]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

import sys
sys.path.append('drive/My Drive/Tesis/repo')

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

### Especificaciones de la GPU utilizada

In [0]:
!nvidia-smi

## Configuración del modelo

### Importación de librerías necesarias

In [0]:
import tensorflow as tf
from src.neural_network.ZorzNet.ZorzNetData import ZorzNetData
from src.utils.Database import Database
from src.utils.ClassicLabel import ClassicLabel
from src.utils.ProjectData import ProjectData
import time
from src.neural_network.ZorzNet.ZorzNetData import ZorzNetData

### Importación de las funciones que definen cómo leer el dataset y la función que define el modelo de red

In [0]:
from src.Estimators.zorznet.data_input_fn import data_input_fn
from src.Estimators.zorznet.model_fn import model_fn

### Definición de los hiperparámetros de la red



In [0]:
project_data = ProjectData()

network_data = ZorzNetData()
network_data.model_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_MODEL_PATH
network_data.checkpoint_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_CHECKPOINT_PATH
network_data.tensorboard_path = 'drive/My Drive/Tesis/repo/' + project_data.ZORZNET_TENSORBOARD_PATH

network_data.num_classes = ClassicLabel.num_classes - 1
network_data.num_features = 494
network_data.noise_stddev = 0.1
network_data.num_reduce_by_half = 0

network_data.num_dense_layers_1 = 1
network_data.num_units_1 = [400] * network_data.num_dense_layers_1
network_data.dense_activations_1 = [tf.nn.relu] * network_data.num_dense_layers_1
network_data.batch_normalization_1 = True
network_data.batch_normalization_trainable_1 = True
network_data.keep_prob_1 = [0.6] * network_data.num_dense_layers_1
network_data.kernel_init_1 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_1
network_data.bias_init_1 = [tf.zeros_initializer()] * network_data.num_dense_layers_1

network_data.is_bidirectional = True
network_data.num_cell_units = [512] * 2
network_data.cell_activation = [tf.nn.tanh] * 2
network_data.keep_prob_rnn = None
network_data.rnn_batch_normalization = False

network_data.num_dense_layers_2 = 1
network_data.num_units_2 = [150]
network_data.dense_activations_2 = [tf.nn.relu] * network_data.num_dense_layers_2
network_data.batch_normalization_2 = True
network_data.batch_normalization_trainable_2 = True
network_data.keep_prob_2 = [0.6]
network_data.kernel_init_2 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_2
network_data.bias_init_2 = [tf.zeros_initializer()] * network_data.num_dense_layers_2

network_data.dense_regularizer = 0.5
network_data.rnn_regularizer = 0.0

network_data.beam_width = 0    # 0 -> greedy_decoder, >0 -> beam_search

network_data.learning_rate = 0.001
network_data.use_learning_rate_decay = True
network_data.learning_rate_decay_steps = 5000
network_data.learning_rate_decay = 0.98

network_data.clip_gradient = 5
network_data.optimizer = 'adam'      # 'rms', 'adam', 'momentum', 'sgd'
network_data.momentum = None

### Configuración de entrenamiento

In [0]:
model_dir = 'drive/My Drive/Tesis/repo/out/zorznet/estimator/'

base_path = 'drive/My Drive/classic/'
base_path2 = 'drive/My Drive/classic2/'

from random import shuffle
index_files = [i for i in range(1,51,1)]
index_files2 = [i for i in range(51,93,1)]

train_files = ['train_database_{}.tfrecords'.format(item) for item in index_files]
train_files2 = ['train_database_{}.tfrecords'.format(item) for item in index_files2]

train_files = list(map(lambda x: base_path + x, train_files))
train_files2 = list(map(lambda x: base_path2 + x, train_files2))
train_files = train_files + train_files2

shuffle(train_files)
print(train_files)

val_files = ['test_database_1.tfrecords', 'test_database_2.tfrecords']
test_files = ['test_database_1.tfrecords']

val_files = list(map(lambda x: base_path + x, val_files))
test_files = list(map(lambda x: base_path + x, test_files))

train_batch_size = 40
train_epochs = 1000

validate_batch_size = 40
               

config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_checkpoints_steps=50,
    save_summary_steps=50,
    log_step_count_steps=50)


model = tf.estimator.Estimator(
    model_fn=model_fn,
    params=network_data.as_dict(),
    config=config
)

tf.logging.set_verbosity(tf.logging.INFO)

## Ejecución de la red

### Entrenamiento

In [0]:
model.train(
    input_fn=lambda: data_input_fn(
        filenames=train_files,
        batch_size=train_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=10,
        num_features=network_data.num_features,
        num_epochs=train_epochs)
)


### Validación


In [0]:
model.evaluate(
    input_fn=lambda: data_input_fn(
        filenames=val_files,
        batch_size=validate_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features)
)

### Testeo

In [0]:
num_tests = 5

predictions = model.predict(
    input_fn=lambda: data_input_fn(
        filenames=test_files,
        batch_size=1,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features)
)

for item in predictions:
  print("Predicted: " + ClassicLabel.from_index(item))
  num_tests -= 1
  if num_tests == 0:
    break